In [69]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer
import os

In [70]:
display (os.getcwd())

'F:\\Acmegrade__ML\\Data_file_folder'

In [71]:
os.chdir('F:\\Acmegrade__ML\\Data_file_folder')
movies_data =pd.read_csv('movies.csv')
movies_data

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [72]:
display (movies_data.shape)
display (movies_data.info())

(4803, 24)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

None

In [73]:
display (movies_data.isna().sum())

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [74]:
selected_features = ['genres','keywords','overview','popularity','cast','director']
print(selected_features)

['genres', 'keywords', 'overview', 'popularity', 'cast', 'director']


In [75]:
display (movies_data[selected_features])

,genres,keywords,overview,popularity,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",150.437577,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",139.082615,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A cryptic message from Bond’s past sends him o...,107.376788,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,112.312950,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",43.926995,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,El Mariachi just wants to play his guitar and ...,14.269792,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,Comedy Romance,NaN,A newlywed couple's honeymoon is upended by th...,0.642552,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,NaN,NaN,When ambitious New York attorney Sam is sent t...,0.857008,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


In [76]:
display (movies_data[selected_features].isna().sum())

genres         28
keywords      412
overview        3
popularity      0
cast           43
director       30
dtype: int64

In [77]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer = imputer.fit(movies_data[selected_features])
movies_data[selected_features] = imputer.transform(movies_data[selected_features])
movies_data[selected_features]

,genres,keywords,overview,popularity,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",150.437577,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",139.082615,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A cryptic message from Bond’s past sends him o...,107.376788,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,112.31295,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",43.926995,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,El Mariachi just wants to play his guitar and ...,14.269792,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,Comedy Romance,independent film,A newlywed couple's honeymoon is upended by th...,0.642552,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,Drama,independent film,When ambitious New York attorney Sam is sent t...,0.857008,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


In [78]:
display(movies_data[selected_features].isna().sum())

genres        0
keywords      0
overview      0
popularity    0
cast          0
director      0
dtype: int64

In [79]:
movies_data['popularity'] = str(movies_data['popularity'])
display(movies_data['popularity'].dtype)

dtype('O')

In [80]:
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['overview']+' '+movies_data['popularity']+' '+movies_data['cast']+' '+movies_data['director']

vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
display (feature_vectors.shape)
print(feature_vectors)


(4803, 30108)

  (0, 4226)	0.14002654230107148
  (0, 13952)	0.0868608366930776
  (0, 22793)	0.15883441095538173
  (0, 17513)	0.13019965225305838
  (0, 15327)	0.18160644088994332
  (0, 25484)	0.122834649790705
  (0, 29157)	0.1596516398816397
  (0, 24438)	0.16842214725303634
  (0, 23232)	0.17806013492710704
  (0, 30061)	0.1596516398816397
  (0, 29670)	0.19334056642542258
  (0, 23273)	0.11707494545854176
  (0, 19031)	0.026557844512324805
  (0, 8186)	0.026557844512324805
  (0, 284)	0.026557844512324805
  (0, 15625)	0.026557844512324805
  (0, 20722)	0.026557844512324805
  (0, 18408)	0.026557844512324805
  (0, 345)	0.026557844512324805
  (0, 283)	0.026557844512324805
  (0, 332)	0.026557844512324805
  (0, 282)	0.026557844512324805
  (0, 274)	0.026557844512324805
  (0, 281)	0.026557844512324805
  (0, 307)	0.026557844512324805
  :	:
  (4802, 283)	0.01588078672731064
  (4802, 332)	0.01588078672731064
  (4802, 282)	0.01588078672731064
  (4802, 274)	0.01588078672731064
  (4802, 281)	0.01588078672731064
  (4802, 

In [81]:
similarity = cosine_similarity(feature_vectors)
print  (similarity )

print(similarity.shape)

sim = pd.DataFrame (similarity)
sim

[[1.         0.05112792 0.04979627 ... 0.0326768  0.04462413 0.01834445]
 [0.05112792 1.         0.04605329 ... 0.04664051 0.03293886 0.02421866]
 [0.04979627 0.04605329 1.         ... 0.03921088 0.06337291 0.02165728]
 ...
 [0.0326768  0.04664051 0.03921088 ... 1.         0.04758332 0.04410073]
 [0.04462413 0.03293886 0.06337291 ... 0.04758332 1.         0.03871975]
 [0.01834445 0.02421866 0.02165728 ... 0.04410073 0.03871975 1.        ]]
(4803, 4803)


,0,1,2,3,4,5,6,7,8,9,...,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802
0,1.000000,0.051128,0.049796,0.044768,0.095085,0.070362,0.028245,0.081997,0.039403,0.042706,...,0.022579,0.024514,0.048872,0.057905,0.032341,0.042552,0.025097,0.032677,0.044624,0.018344
1,0.051128,1.000000,0.046053,0.036585,0.079640,0.060613,0.031122,0.074632,0.048475,0.051083,...,0.019144,0.031443,0.042903,0.048396,0.054793,0.038915,0.028003,0.046641,0.032939,0.024219
2,0.049796,0.046053,1.000000,0.051201,0.050903,0.042038,0.023307,0.091715,0.054222,0.042160,...,0.033779,0.017815,0.022332,0.024575,0.039511,0.036291,0.021849,0.039211,0.063373,0.021657
3,0.044768,0.036585,0.051201,1.000000,0.040757,0.038579,0.042178,0.068239,0.046074,0.134009,...,0.026745,0.024738,0.042403,0.044572,0.033948,0.041883,0.023562,0.056657,0.058721,0.033330
4,0.095085,0.079640,0.050903,0.040757,1.000000,0.080987,0.049167,0.099466,0.039705,0.056886,...,0.027467,0.040145,0.041356,0.052110,0.068410,0.044849,0.023313,0.044890,0.041457,0.027815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,0.042552,0.038915,0.036291,0.041883,0.044849,0.028780,0.030359,0.062981,0.042430,0.028436,...,0.017698,0.027581,0.041488,0.024958,0.056944,1.000000,0.014995,0.025293,0.041234,0.026654
4799,0.025097,0.028003,0.021849,0.023562,0.023313,0.023714,0.020105,0.023093,0.023878,0.018411,...,0.019881,0.034056,0.019600,0.026298,0.025883,0.014995,1.000000,0.033370,0.036477,0.016134
4800,0.032677,0.046641,0.039211,0.056657,0.044890,0.052128,0.045959,0.043340,0.049084,0.037366,...,0.028351,0.032156,0.047380,0.059849,0.032029,0.025293,0.033370,1.000000,0.047583,0.044101
4801,0.044624,0.032939,0.063373,0.058721,0.041457,0.044204,0.037829,0.038268,0.072097,0.028690,...,0.017360,0.039538,0.036913,0.039333,0.048450,0.041234,0.036477,0.047583,1.000000,0.038720


In [84]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('The top 20 movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<21):
    print(i, '.',title_from_index)
    i+=1


 Enter your favourite movie name : King Kong
The top 20 movies suggested for you : 

1 . King Kong
2 . Morning Glory
3 . America's Sweethearts
4 . Meet Dave
5 . All That Jazz
6 . The Pianist
7 . What Just Happened
8 . Thank You for Smoking
9 . Punch-Drunk Love
10 . Mutual Friends
11 . My Last Day Without You
12 . Big Fat Liar
13 . Redbelt
14 . Get Shorty
15 . Someone Like You...
16 . Boogie Nights
17 . Last Action Hero
18 . Unfinished Business
19 . The Black Hole
20 . The Last Shot
